In [1]:
import pandas as pd
import numpy as np

In [2]:
INPUT_FILE = '../dataset/flipkart_products_preprocessed_2.csv'

In [3]:
df = pd.read_csv(INPUT_FILE)

In [4]:
df["text"] = (
    df["title"].fillna("") + " " +
    df["description"].fillna("") + " " +
    df["level_combined"].fillna("")
)

In [5]:
features = [
    "price",
    "rating_value",
    "level_2",
    "level_3", 
    "text"
]

X = df[features]
y = df["discount"]

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
numeric_features = [
    "price",
    "rating_value"
]

categorical_features = [
    "level_2",
    "level_3"
]

text_features = [
    "text"
]

In [8]:
numeric_transformer = StandardScaler()
categorical_transformer = TargetEncoder()

text_tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("text_tfidf", text_tfidf, "text")
    ],
    remainder="drop"
)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
pipeline = Pipeline(
    steps = [ 
        ("preprocess", preprocessor),
        ("model", DecisionTreeRegressor())
    ]        
)

In [12]:
pipeline.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [13]:
y_pred = pipeline.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R^2:", r2)

RMSE: 14.464787473711729
R^2: 0.438499105208057
